In [ ]:
import numpy as np
from typing import Dict, List
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from celluloid import Camera
import time 
from IPython.display import HTML

In [ ]:
class Grid:
    def __init__(self, max_x: int, max_y: int) -> None:
        self.gridxsize = max_x
        self.gridysize = max_y
        self.generate_coords()
        self.animals_in_grid = [[[] for _ in range(max_y)] for _ in range(max_x)]
        self.directions = ["LEFT", "RIGHT", "UP", "DOWN", "STAY"]
        
    def generate_coords(self) -> None:
        xcoords = [i for i in range(self.gridxsize)]
        ycoords = [i for i in range(self.gridysize)]
        random.shuffle(xcoords)
        random.shuffle(ycoords) 
        self.coords = [[x,y] for x in xcoords for y in ycoords]
        random.shuffle(self.coords)

In [ ]:
class Species():
    def __init__(self, PredatorOrPrey: str, E_0: int, E_procreation: float, N: int, Grid_instance: object) -> None:
        self.pred_or_prey = PredatorOrPrey
        self.initial_energy = E_0
        self.min_procreation_energy = E_procreation
        self.num_agents = N
        self.agents : List[object] = []
        self.babies : List[object] = []
        self.death_list : List[object] = []
        
        self.grid = Grid_instance
        random.shuffle(self.grid.coords)
        self.species_coords = self.grid.coords[:self.num_agents]        
        
    def move(self):
        # Reset grids
        self.species_coords = [[0, 0]] * len(self.agents)        
        
        for i, agent in enumerate(self.agents):
            dir_index = random.randint(0, 4)
            agent.move(self.grid.directions[dir_index])
            
            self.species_coords[i] = [agent.x, agent.y]
            agent.agent_index = i

            y_or_d_condition = ('y' if self.pred_or_prey == "Prey" else 'd')
            self.grid.animals_in_grid[agent.x][agent.y].append(f'{y_or_d_condition}{i}')

In [ ]:
class Agent(Species):
    
    def __init__(self, species: object, agent_number: int, energy=None) -> None:
        super().__init__(species.pred_or_prey, 
                         species.initial_energy, 
                         species.min_procreation_energy, 
                         species.num_agents, 
                         species.grid)
        self.parent_species = species

        y_or_d_condition = ('y' if self.pred_or_prey == "Prey" else 'd')
        self.agent_id = f"{y_or_d_condition}{agent_number}"
        self.agent_index = agent_number

        self.x = self.parent_species.species_coords[agent_number][0]
        self.y = self.parent_species.species_coords[agent_number][1]

        if energy is not None:  self.energy = energy
        else: self.energy = np.random.normal(species.initial_energy, species.initial_energy/3)

        if self.parent_species.pred_or_prey == "Prey":  
              self.step_size = 1
        else: self.step_size = random.choice([1,2,2,3])
            
    def add_to_death_list(self):
        self.parent_species.death_list.append(self)
        
    def procreate(self, other):

        Baby = Agent(self.parent_species, 
                     self.agent_index, 
                     energy=(self.energy + other.energy)/2)

        self.energy = 3 * self.energy / 4
        other.energy = 3 * other.energy / 4

        self.parent_species.babies.append(Baby)
   
    
    def move(self, direction : str):
        step = round(self.step_size * self.energy / 100 + 0.6)
        # Change agent x and y coords
        self.energy -=step
        if direction == "LEFT":
            self.x += -step if self.x - step > 0 else +step
        if direction == "RIGHT":
            self.x += step if self.x + step < self.grid.gridxsize-1 else -step
        if direction == "UP":
            self.y += step if self.y + step < self.grid.gridysize-1 else -step
        if direction == "DOWN":
            self.y += -step if self.y - step > 0 else +step
        if direction == "STAY":
            if self.pred_or_prey == "Prey": self.energy += 10
            if self.pred_or_prey == "Pred": self.energy += 4

        # Kill if no energy
        if self.energy <= 0: self.add_to_death_list() 

In [ ]:
def initialise_agents(species_instance: object):
    for i in range(species_instance.num_agents):
        this_agent = Agent(species_instance, i)
        species_instance.agents.append(this_agent)
        agent_x = species_instance.species_coords[i][0]
        agent_y = species_instance.species_coords[i][1]
        y_or_d_condition = ('y' if species_instance.pred_or_prey == "Prey" else 'd')
        species_instance.grid.animals_in_grid[agent_x][agent_y] = [f'{y_or_d_condition}{i}']

def filter_negative_pairs(array):
  xs, ys = [], []
  for pair in array:
    if pair[0] >= 0 and pair[1] >= 0:
      xs.append(int(pair[0]))
      ys.append(int(pair[1]))
  return xs, ys

def kill_death_list(species_object):

    death_indices = [slain.agent_index for slain in species_object.death_list]
    # print(death_indices)
    for index in sorted(set(death_indices), reverse=True):
        del species_object.agents[index]
        del species_object.species_coords[index]
    
    species_object.num_agents-=len(death_indices)
    species_object.death_list : List[object] = []

def plot_frame(fig, ax, Prey, Pred):
    gridxsize, gridysize = Prey.grid.gridxsize, Prey.grid.gridysize
    x_prey, y_prey = filter_negative_pairs(Prey.species_coords)
    x_pred, y_pred = filter_negative_pairs(Pred.species_coords)
    prey_scatter = ax.scatter(x_prey, y_prey, c='green')
    pred_scatter = ax.scatter(x_pred, y_pred, c='red')
    ax.set(xlim=(0, gridxsize), ylim=(0, gridysize))
    plt.axis('off')
    return fig, ax

def timeme(method):
    def wrapper(*args, **kw):
        startTime = int(round(time.time() * 1000))
        result = method(*args, **kw)
        endTime = int(round(time.time() * 1000))
        print(endTime - startTime,'ms')
        return result
    return wrapper

@timeme
def interact(Prey_species_obj, Pred_species_obj, Grid_obj):
    for y_i in range(Grid_obj.gridysize):
        for x_i in range(Grid_obj.gridxsize):

            if not Grid_obj.animals_in_grid[y_i][x_i]: pass
            preys, preds = [], []
            
            for agent_str in Grid_obj.animals_in_grid[y_i][x_i]:
                if agent_str[0] == 'y': preys.append(int(agent_str[1:]))
                if agent_str[0] == 'd': preds.append(int(agent_str[1:]))

                # Predators procreate
                for i in range(0, len(preds)-1, 2):
                    pred_mum = Pred_species_obj.agents[preds[i]]
                    pred_dad = Pred_species_obj.agents[preds[i+1]]
                    pred_mum.energy-=len(preds)
                    pred_dad.energy-=len(preds)
                    if pred_mum.energy + pred_dad.energy > pred_mum.min_procreation_energy:
                        pred_mum.procreate(pred_dad)

                # Predators feed on prey
                if not preys or not preds: pass
                else: 
                    for i in range(len(preds)):
                        if i < len(preys):
                            predator = Pred_species_obj.agents[preds[i]]
                            food = Prey_species_obj.agents[preys[i]]
                            predator.energy+=food.energy
                            food.add_to_death_list()
                            
            # Preys procreate
                for i in range(0, len(preys)-1, 2):
                    prey_mum = Prey_species_obj.agents[preys[i]]
                    prey_dad = Prey_species_obj.agents[preys[i+1]]
                    prey_mum.energy-=len(preys)
                    prey_dad.energy-=len(preys)
                    if prey_mum.energy + prey_dad.energy > prey_mum.min_procreation_energy:
                        prey_mum.procreate(prey_dad)   
 

    # At the end of the turn make babies into adults

    for baby in Prey_species_obj.babies:
        Prey_species_obj.agents.append(baby)
    Prey_species_obj.babies : List[object] = []

    
    for baby in Pred_species_obj.babies:
        Pred_species_obj.agents.append(baby)
    Pred_species_obj.babies : List[object] = []

    kill_death_list(Prey_species_obj)
    kill_death_list(Pred_species_obj)


In [ ]:
gridxsize = 100
gridysize = 100

Prey_E0 = 200
Prey_EP = 40
Prey_N = 600

Pred_E0 = 50
Pred_EP = 20
Pred_N = 220
    
TheGrid = Grid(gridxsize, gridysize)
Prey = Species("Prey", Prey_E0, Prey_EP, Prey_N, TheGrid)
Pred = Species("Predator", Pred_E0, Pred_EP, Pred_N, TheGrid)
initialise_agents(Prey)
initialise_agents(Pred)

fig, ax = plt.subplots()
camera = Camera(fig)

for i in range(1000):
    TheGrid.animals_in_grid = [[[] for _ in range(TheGrid.gridxsize)] for _ in range(TheGrid.gridysize)]

    Prey.move()
    Pred.move()
    interact(Prey, Pred, TheGrid)
    print(i, len(Prey.agents), len(Pred.agents))

    fig, ax = plot_frame(fig, ax, Prey, Pred)
    camera.snap()

plt.close()
animation = camera.animate(interval = 35)
animation.save('predatorprey.mp4')
HTML(animation.to_html5_video())